# LSTM을 사용한 영화 리뷰 감정 분석

영화 리뷰와 같은 데이터는 일반적인 수치 데이터와 달리 사람의 언어, 즉 자연어 데이터입니다. 자연어 데이터는 머신러닝에서 특히 다루기 까다로운 데이터인데요. 데이터 자체가 길고 전처리할 부분이 많을 뿐만 아니라 순서나 단어의 유사성 개념을 이해해야만 하기 때문입니다.

딥러닝에서는 이렇듯 순서가 있는 데이터를 처리하기 위해 RNN (Recurrent Neural Network)를 사용하고, 일반적인 RNN에서 발생하는 문제를 막기 위한 다양한 종류의 개량된 레이어를 사용합니다.

단어의 유사성을 학습하는 것은, 단어를 단순히 1, 2, 53과 같은 숫자로 표현하는 대신 [0.3, 0.4, 2.1] 과 같은 좌표로 표현하는 것을 통해 가능합니다. 좌표 공간에서의 방향이나 거리를 인간이 이해하는 단어 사이의 관계를 표현하기 위해 쓰는 것이죠. 이를 word embedding이라고 하며 자연어 처리 분야에서 거의 필수적인 요소입니다.

LSTM(RNN) 소개  
https://brunch.co.kr/@chris-song/9

Word Embedding (Word2Vec)  
https://deeplearning4j.org/kr/word2vec

In [ ]:
# Keras의 백엔드 프레임워크로 Tensorflow를 사용합니다
import tensorflow as tf

# 실습을 진행하기 위해 선생님 그래픽카드 중 어떤 카드를 쓸지
# 해당 카드의 GPU 메모리를 몇 % 사용할지 설정하는 부분입니다
# 실습을 위한 것이므로 일반적인 환경에서는 필요 없습니다 (몰라도 됨)
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.1
config.gpu_options.visible_device_list = "0"
set_session(tf.Session(config=config))

# 이 셀을 실행하고 *이 사라진 것을 확인 후 다음으로 진행하세요

In [ ]:
# LSTM for sequence classification in the IMDB dataset
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM, CuDNNLSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

# load the dataset but only keep the top n words, zero the rest
top_words = 3000
index_from_num = 3

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words, index_from=index_from_num)

# truncate and pad input sequences
max_review_length = 300
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

X_alt_train = X_train[..., numpy.newaxis]
X_alt_test = X_test[..., numpy.newaxis]
X = X_train
X_t = X_test

In [ ]:
print(X_t[0])

In [ ]:
# To see original text data
(X_original_train, y_original_train), (X_original_test, y_original_test) = imdb.load_data(index_from=index_from_num)
word_to_id = imdb.get_word_index()
word_to_id = {k:(v+index_from_num) for k,v in word_to_id.items()}
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<UNK>"] = 2
id_to_word = {value:key for key,value in word_to_id.items()}

In [ ]:
def dense_only():
    global X
    global X_t
    X = X_train
    X_t = X_test
    model = Sequential()
    model.add(Dense(512, input_shape=(max_review_length, )))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
def simple():
    global X
    global X_t
    X = X_alt_train
    X_t = X_alt_test
    model = Sequential()
    model.add(LSTM(128, input_shape=(max_review_length, 1)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
def simple_cuDNN():
    global X
    global X_t
    X = X_alt_train
    X_t = X_alt_test
    model = Sequential()
    model.add(CuDNNLSTM(128, input_shape=(max_review_length, 1)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
def stacked():
    global X
    global X_t
    X = X_alt_train
    X_t = X_alt_test
    model = Sequential()
    model.add(CuDNNLSTM(128, input_shape=(max_review_length, 1), return_sequences=True))
    model.add(CuDNNLSTM(128))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
embedding_vector_length = 32
def embedding():
    global X
    global X_t
    X = X_train
    X_t = X_test
    model = Sequential()
    model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length))
    model.add(CuDNNLSTM(128))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
embedding_vector_length = 32
def embedding_stacked():
    global X
    global X_t
    X = X_train
    X_t = X_test
    model = Sequential()
    model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length))
    model.add(CuDNNLSTM(128, return_sequences=False))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
model = dense_only()
print(model.summary())

In [ ]:
model.fit(X, y_train, epochs=5, batch_size=32)

In [ ]:
# Final evaluation of the model
scores = model.evaluate(X_t, y_test, batch_size=32, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
start_index = 0
end_index = 10
predict = model.predict(X_t[start_index:end_index+1])

for i, p in enumerate(predict):
    if (p[0] < 0.5 and y_test[i] == 0) or (p[0] > 0.5 and y_test[i] == 1):
        correct = '맞음'
    else:
        correct = '틀림'
    print("%d번 데이터 (%s) - 예측: %.3f / 실제: %d" % (start_index + i, correct, p[0], y_test[i]))
    print(' '.join(id_to_word[id] for id in X_original_test[i] if id != 0 and id != 1 ))
    print('')